In [1]:
import os
import pandas as pd
import json
import requests
import numpy as np
import re

In [ ]:
#s.chdir('../')

In [8]:
#review_path = 'data/yelp_academic_dataset_review.json'
#business_path = 'data/yelp_academic_dataset_business.json'
#review_path_out = 'review.csv'
#business_path_out = 'business.csv'

### Load and preprocess raw data into firebase 

In [2]:
def clear_firebase_cache():
    requests.delete(url = 'https://project-3e24f-default-rtdb.firebaseio.com/.json')
    return

In [3]:
def get_info(df, path):
    result_dict = {}
    result_dict['file name'] = os.path.basename(path)
    result_dict['file size'] = str(int(os.path.getsize(path) / 1024**2)) + ' MB' 
    result_dict['number of attributes'] = len(df.columns)
    result_dict['number of rows'] = len(df)
    result_dict['attributes name'] = df.columns.values.tolist()
    return result_dict

In [60]:
def json_to_csv(review_path_in, business_path_in, test = True):
    '''convert the json file into the csv file we needed'''

    print("----- Start Processing Raw Review Json file -----")
    review_data = {"review_id":[],"business_id":[],"text":[],"stars":[]}
    with open(review_path_in, encoding="utf8") as fi:
        for line in fi:
            review = json.loads(line)
            review_data['review_id'].append(review['review_id'])
            review_data['business_id'].append(review['business_id'])
            review_data['text'].append(review['text'])
            review_data['stars'].append(review['stars'])
    review_df = pd.DataFrame(review_data)
    print(review_df.head(5))
    print("----- Finish cleaning up Review Json file-----")
    
    print("----- Start Processing Raw Business Json file-----")
    business_data = {"name":[],"business_id":[],"city":[],"categories":[], "address":[], "review_count":[], "hours":[]}
    with open(business_path_in, encoding="utf8") as f:
        for line in f:
            business = json.loads(line)
            business_data['name'].append(business['name'])
            business_data['business_id'].append(business['business_id'])
            business_data['city'].append(business['city'])
            business_data['categories'].append(business['categories'])
            business_data['address'].append(business['address'])
            business_data['review_count'].append(business['review_count'])
            business_data['hours'].append(business['hours'])
    business_df = pd.DataFrame(business_data)
    print(business_df.head(5))
    print("----- Finish cleaning up Business Json file-----")
    
    if test:
        test_r_df = review_df.head(1000)
        test_b_df = business_df[(business_df['business_id']).isin(test_r_df['business_id'])]
        requests.put(url = 'https://project-3e24f-default-rtdb.firebaseio.com/review.json', data = test_r_df.to_json(orient = 'records'))
        requests.put(url = 'https://project-3e24f-default-rtdb.firebaseio.com/business.json', data = test_b_df.to_json(orient = 'records'))
        print("----- Finish Uploading Test Business and Review to Firebase -----")
    else:
        requests.put(url = 'https://project-3e24f-default-rtdb.firebaseio.com/review.json', data = review_df.to_json())
        requests.put(url = 'https://project-3e24f-default-rtdb.firebaseio.com/business.json', data = business_df.to_json())
        print("----- Finish Uploading Business and Review to Firebase -----")
    
    
    requests.put(url = 'https://project-3e24f-default-rtdb.firebaseio.com/review_info.json', 
                 data = json.dumps(get_info(review_df, review_path_in)))
    requests.put(url = 'https://project-3e24f-default-rtdb.firebaseio.com/business_info.json',
                 data = json.dumps(get_info(business_df, business_path_in)))
    print("----- Finish Uploading File Information to Firebase -----")
    
               
    return

In [102]:
review_path = 'data/yelp_academic_dataset_review.json'
business_path = 'data/yelp_academic_dataset_business.json'

In [103]:
clear_firebase_cache()
json_to_csv(review_path, business_path)

----- Start Processing Raw Review Json file -----
                review_id             business_id  \
0  lWC-xP3rd6obsecCYsGZRg  buF9druCkbuXLX526sGELQ   
1  8bFej1QE5LXp4O05qjGqXA  RA4V8pr014UyUbDvI-LW2A   
2  NDhkzczKjLshODbqDoNLSg  _sS2LBIGNT5NQb6PD1Vtjw   
3  T5fAqjjFooT4V0OeZyuk1w  0AzLzHfOJgL7ROwhdww2ew   
4  sjm_uUcQVxab_EeLCqsYLg  8zehGz9jnxPqXtOc7KaJxA   

                                                text  stars  
0  Apparently Prides Osteria had a rough summer a...    4.0  
1  This store is pretty good. Not as great as Wal...    4.0  
2  I called WVM on the recommendation of a couple...    5.0  
3  I've stayed at many Marriott and Renaissance M...    2.0  
4  The food is always great here. The service fro...    4.0  
----- Finish cleaning up Review Json file-----
----- Start Processing Raw Business Json file-----
                      name             business_id         city  \
0      Oskar Blues Taproom  6iYb2HFDywm3zjuRg0shjw      Boulder   
1  Flying Elephants at PDX 

In [92]:
k = requests.get(url = 'https://project-3e24f-default-rtdb.firebaseio.com/review_info.json').json()

### get processed file information

In [231]:
spark = SparkSession.builder.appName('project').getOrCreate()
review = json.loads(data.text)

In [220]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [104]:
url = 'https://project-3e24f-default-rtdb.firebaseio.com/'
city = 'Austin'

In [105]:
business = requests.get(url = url + 'business.json?orderBy="city"&equalTo="'+str(city)+'"').json()
data = requests.get(url = 'https://project-3e24f-default-rtdb.firebaseio.com/review.json').json()

In [106]:
city_business = pd.DataFrame.from_dict(business, orient="index")
city_rest = city_business[(city_business.categories.str.contains('Restaurants'))| (city_business.categories.str.contains('Food'))]

In [107]:
review_df = pd.json_normalize(data)
city_review = review_df[review_df['business_id'].isin(city_rest['business_id'].tolist())]

In [115]:
def preprocess_group_restaurant_review(review_df, business_df):
    restaurant_df = review_df[['text','stars','business_id']].groupby('business_id').agg({'text':list,
                                                                                          'stars':np.mean}
                                                                                        ).reset_index()
    restaurant_df.text = restaurant_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return pd.merge(restaurant_df, business_df.reset_index(), on ='business_id')

In [116]:
grouped_rest = preprocess_group_restaurant_review(city_review, city_rest)
grouped_rest

,business_id,text,stars,index,address,categories,city,hours,name,review_count
0,-1ShItlulHnBsoOQWnblzw,I was super excited to finally find a place ...,2.0,675,7309 McNeil Dr,"Food Trucks, Mexican, Food, Restaurants",Austin,"{'Friday': '11:0-22:0', 'Monday': '11:0-22:0',...",Super Burrito at the Water Tank,105
1,-IIvmjoEKa9Trhf4OxzJeA,This place is amazing From the staff to th...,5.0,512,621 Congress Ave,"Bakeries, Food, Breakfast & Brunch, American (...",Austin,"{'Friday': '16:0-22:0', 'Saturday': '7:0-22:0'...",Caroline,770
2,-L69Ix0-xX4BlHA61fGvrQ,This Chicken Express has been doing it right...,5.0,611,"8300 N Fm 620, Bldg H","Fast Food, Restaurants, Food, Juice Bars & Smo...",Austin,"{'Friday': '10:30-22:0', 'Monday': '10:30-22:0...",Chicken Express,73
3,-_GnwXmzC3DXsHR9nyaC2g,3 5 stars I got the avocado margarita and i...,4.0,535,614 E Oltorf St,"Restaurants, Mexican",Austin,"{'Friday': '9:0-22:0', 'Monday': '11:0-21:0', ...",Curra's Grill,1148
4,-oCYfB-JL3Yc_RECg4B1Qw,I have mixed feelings about this place Firs...,3.0,563,212 E 6th St,"Wedding Chapels, Donuts, Food, Bakeries, Event...",Austin,"{'Friday': '8:0-0:0', 'Monday': '8:0-0:0', 'Sa...",Voodoo Doughnut - 6th Street,1809
...,...,...,...,...,...,...,...,...,...,...
81,uv1Fl6F7I9KRONRCg2pKpQ,I have ventured over to Wahoo s a few times ...,3.0,562,1722 South Congress,"Restaurants, Mexican, Hawaiian, Seafood, Food,...",Austin,"{'Friday': '11:0-22:0', 'Monday': '11:0-21:0',...",Wahoo's Fish Tacos,114
82,v1UzkU8lEWdjxq8byWFOKg,Delish n nThe only downside is that it is d...,5.0,786,117 San Jacinto Blvd,"Soul Food, Southern, Comfort Food, Chicken Sho...",Austin,"{'Friday': '11:0-22:0', 'Monday': '0:0-0:0', '...",Gus's World Famous Fried Chicken,2490
83,vJUTrk0-nm7JbNzPVcPo4A,At first i was looking at this dish strange ...,4.0,544,2900 W Anderson Ln,"Japanese, Sushi Bars, Restaurants",Austin,"{'Friday': '11:0-21:0', 'Monday': '11:0-21:0',...",Zen Japanese Food Fast,250
84,vzSXdDYtnm_F_x1hY2UVVw,Authentic Not fancy n nKnow what you re...,4.0,593,8650 Spicewood Springs Rd,"Noodles, Restaurants, Chinese",Austin,"{'Friday': '11:0-21:0', 'Monday': '11:0-21:0',...",Chen's Noodle House,471


In [119]:
requests.put(url = 'https://project-3e24f-default-rtdb.firebaseio.com/grouped_df.json', data = grouped_rest.to_json(orient = 'records'))

<Response [200]>

In [121]:
k = requests.get(url = 'https://project-3e24f-default-rtdb.firebaseio.com/grouped_df.json').json()

In [123]:
tt= pd.json_normalize(k)

### TF-IDF on the grouped restaurant data

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [124]:
%%time
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.1, stop_words='english')
tfidf_matrix = tf.fit_transform(tt.text.apply(lambda x: x.lower()))

Wall time: 33.5 ms


In [125]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [132]:
rest_rest_df = pd.DataFrame(cosine_sim, columns=tt.business_id, index=tt.business_id)
rest_rest_df

business_id,-1ShItlulHnBsoOQWnblzw,-IIvmjoEKa9Trhf4OxzJeA,-L69Ix0-xX4BlHA61fGvrQ,-_GnwXmzC3DXsHR9nyaC2g,-oCYfB-JL3Yc_RECg4B1Qw,1diwr6ycCvtBXR5vmnmNGg,2faLQhuvFdk-fHAg9lWXeA,2hPkrbzphCtP_e2neIyBPg,4THc7ojs6ppNHKhLl74www,4cgE4DUDSH3CjqsRwX_LOA,...,o8DMdE-VAiUydUBJ2E6MZA,oug5bLTWP_YTtj1C3_X6Xw,pc1PkDBggUSNU73jONdqHw,qDyZQ6mT1JajQ7_Yj7PeLw,tmvswVDr4n56EjWq_RptBA,uv1Fl6F7I9KRONRCg2pKpQ,v1UzkU8lEWdjxq8byWFOKg,vJUTrk0-nm7JbNzPVcPo4A,vzSXdDYtnm_F_x1hY2UVVw,y5MLZak3tLfiotCHY503bA
business_id,,,,,,,,,,,,,,,,,,,,,
-1ShItlulHnBsoOQWnblzw,1.000000,0.144152,0.111571,0.095089,0.218768,0.313279,0.171415,0.000000,0.124215,0.429312,...,0.202892,0.206747,0.172733,0.466533,0.114296,0.049348,0.086720,0.214663,0.098458,0.000000
-IIvmjoEKa9Trhf4OxzJeA,0.144152,1.000000,0.158529,0.123552,0.020490,0.295833,0.137709,0.150484,0.173398,0.161904,...,0.000000,0.201489,0.289721,0.000000,0.127711,0.154360,0.030204,0.201875,0.000000,0.000000
-L69Ix0-xX4BlHA61fGvrQ,0.111571,0.158529,1.000000,0.188354,0.202336,0.182259,0.474475,0.064485,0.262063,0.183293,...,0.105748,0.137214,0.140787,0.055279,0.211833,0.085956,0.483138,0.164562,0.141644,0.191234
-_GnwXmzC3DXsHR9nyaC2g,0.095089,0.123552,0.188354,1.000000,0.267763,0.112490,0.302554,0.034384,0.208347,0.394622,...,0.000000,0.447595,0.250391,0.000000,0.290039,0.080641,0.248815,0.160660,0.063284,0.000000
-oCYfB-JL3Yc_RECg4B1Qw,0.218768,0.020490,0.202336,0.267763,1.000000,0.254804,0.337446,0.098428,0.350198,0.267071,...,0.401933,0.258944,0.133106,0.132306,0.211682,0.184338,0.192438,0.051634,0.355526,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uv1Fl6F7I9KRONRCg2pKpQ,0.049348,0.154360,0.085956,0.080641,0.184338,0.318274,0.334061,0.215825,0.268019,0.323295,...,0.000000,0.187589,0.438343,0.087795,0.436761,1.000000,0.142774,0.393861,0.256474,0.283845
v1UzkU8lEWdjxq8byWFOKg,0.086720,0.030204,0.483138,0.248815,0.192438,0.088933,0.458374,0.000000,0.000000,0.082244,...,0.000000,0.247054,0.131011,0.000000,0.166761,0.142774,1.000000,0.120616,0.129386,0.101457
vJUTrk0-nm7JbNzPVcPo4A,0.214663,0.201875,0.164562,0.160660,0.051634,0.437332,0.359223,0.124491,0.275675,0.439893,...,0.079970,0.337646,0.298269,0.071559,0.239855,0.393861,0.120616,1.000000,0.477439,0.205941


### make recommendations based on the extracted featrues

In [39]:
def generate_n_dataframe(df, n = 20):
    column = ['business_id']
    for i in range(n):
        column_name = 'Top_' + str(i)
        column.append(column_name)
    # top N recommended restaurant dataframe for all restaurants
    top_df = pd.DataFrame(columns = column)
    # iterate through all restaurants in the dataframe
    for i in df.index:
        ix = df.loc[:,i].to_numpy().argpartition(range(-1,-n,-1))
        high_recomm = df.columns[ix[-1:-(n+2):-1]]
        top_df = top_df.append(pd.Series(high_recomm.values, index = column),ignore_index=True)
    return top_df

In [134]:
result_df = generate_n_dataframe(rest_rest_df, n = 20)
result_df

,business_id,Top_0,Top_1,Top_2,Top_3,Top_4,Top_5,Top_6,Top_7,Top_8,...,Top_10,Top_11,Top_12,Top_13,Top_14,Top_15,Top_16,Top_17,Top_18,Top_19
0,-1ShItlulHnBsoOQWnblzw,NY_oCeK6mqVz-oJfbD0c4A,qDyZQ6mT1JajQ7_Yj7PeLw,4cgE4DUDSH3CjqsRwX_LOA,mOnesB4IF9j6-ZmHoOHOig,OlB0841vj4V7Wje5tTtLWA,C2Sq5A2g7a56wwiqrQ0ucA,PrWSjn4a8o4dHoqKs53GBA,1diwr6ycCvtBXR5vmnmNGg,bZiIIUcpgxh8mpKMDhdqbA,...,KEfjH-T30yTvXssJLjcrlA,mGVVbBW0e6NVyNxHRstz_w,jREzLrIEkc4jQKLfYMJ0gg,hkqFm02Y2ax0Pd1BdEyl1A,C0d5kzUx6C19mLcxQyhxCA,KEh3efWSOIzd8BvuW-X4-g,-oCYfB-JL3Yc_RECg4B1Qw,vJUTrk0-nm7JbNzPVcPo4A,_rt-Z934kfFzgG19nTrIcQ,E9ysHn296GgE9KJg88t83w
1,-IIvmjoEKa9Trhf4OxzJeA,iCpyXPsePXB_UqT4_hVqBg,KEh3efWSOIzd8BvuW-X4-g,_HjtaT4TMHwOoGG_pZmk1A,JCgLWYjoPJlznzFEtnzqKg,hkqFm02Y2ax0Pd1BdEyl1A,g3pXGPRKYd_WQxLT2IFPxQ,KEfjH-T30yTvXssJLjcrlA,1diwr6ycCvtBXR5vmnmNGg,bZiIIUcpgxh8mpKMDhdqbA,...,Mvbfzj05yR5dH397_7XPQw,7CXy5h3u_aWsK9qov6B4rg,LPYACwYAeHJdQIpxEtJYvw,dQ80ktWSRj5UIk96bRUmWw,LUCmOKTK5Uh7eU84yJq3mQ,g-6qv5f2Y4mVrsDD3JRkuQ,GCgnLwwI4b9cOee-vtK1aQ,F6b_O4-wtd3956YOvAoerg,OlB0841vj4V7Wje5tTtLWA,aM-FPdJicti_TzymjW3Usg
2,-L69Ix0-xX4BlHA61fGvrQ,KEfjH-T30yTvXssJLjcrlA,KK-6rXbIo9B4-b_P5WO5Qg,ezX80RabJ62B2lbKXCtg3g,v1UzkU8lEWdjxq8byWFOKg,EwUahd4SYKdPH1p1Cpe5fg,2faLQhuvFdk-fHAg9lWXeA,5_RnnLBKO-5vpb4iiFAvBQ,mOnesB4IF9j6-ZmHoOHOig,P2bibaW6DSwnCpjVXbsZQQ,...,Z-URyf2TJT1Ay-Sp1cVGOw,IbuSlumTCNSD-ohMVHfWUA,9vICKr25A3YG88Zr1wxDiQ,4THc7ojs6ppNHKhLl74www,Dh8R00m1wcCsUcm-Sflmlw,Pkp824mx2LL93TqKOdfMyQ,i1GWpxJ1Phm2awhVaTkouA,Id7w_CCXqh7x1lC3U1-S5w,OlB0841vj4V7Wje5tTtLWA,bZiIIUcpgxh8mpKMDhdqbA
3,-_GnwXmzC3DXsHR9nyaC2g,E9ysHn296GgE9KJg88t83w,oug5bLTWP_YTtj1C3_X6Xw,_Rrpjd7Z7isi8QFAv3kQ2g,4cgE4DUDSH3CjqsRwX_LOA,jREzLrIEkc4jQKLfYMJ0gg,dQ80ktWSRj5UIk96bRUmWw,bZiIIUcpgxh8mpKMDhdqbA,IbuSlumTCNSD-ohMVHfWUA,_jYt69Zx1SUo_V9z0m7Ljg,...,2faLQhuvFdk-fHAg9lWXeA,OlB0841vj4V7Wje5tTtLWA,g-6qv5f2Y4mVrsDD3JRkuQ,9vICKr25A3YG88Zr1wxDiQ,tmvswVDr4n56EjWq_RptBA,NY_oCeK6mqVz-oJfbD0c4A,-oCYfB-JL3Yc_RECg4B1Qw,pc1PkDBggUSNU73jONdqHw,KEfjH-T30yTvXssJLjcrlA,WtDOs3a6k_oPJmwiDh4qBQ
4,-oCYfB-JL3Yc_RECg4B1Qw,_Rrpjd7Z7isi8QFAv3kQ2g,o8DMdE-VAiUydUBJ2E6MZA,dQ80ktWSRj5UIk96bRUmWw,Dh8R00m1wcCsUcm-Sflmlw,SCwzUgW_RUdahXNObknkTg,vzSXdDYtnm_F_x1hY2UVVw,4THc7ojs6ppNHKhLl74www,2faLQhuvFdk-fHAg9lWXeA,jREzLrIEkc4jQKLfYMJ0gg,...,mOnesB4IF9j6-ZmHoOHOig,g-6qv5f2Y4mVrsDD3JRkuQ,khghDJpPqW5hG4IXr8sF4A,gcAXWYT_yoJCE5cAAeaHxQ,9vICKr25A3YG88Zr1wxDiQ,-_GnwXmzC3DXsHR9nyaC2g,bZiIIUcpgxh8mpKMDhdqbA,4cgE4DUDSH3CjqsRwX_LOA,_jYt69Zx1SUo_V9z0m7Ljg,oug5bLTWP_YTtj1C3_X6Xw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,uv1Fl6F7I9KRONRCg2pKpQ,bZiIIUcpgxh8mpKMDhdqbA,LPYACwYAeHJdQIpxEtJYvw,pc1PkDBggUSNU73jONdqHw,tmvswVDr4n56EjWq_RptBA,OlB0841vj4V7Wje5tTtLWA,Id7w_CCXqh7x1lC3U1-S5w,bWfbGdWSfRFLswxnoMt5oA,dQ80ktWSRj5UIk96bRUmWw,YZs1gNSh_sN8JmN_nrpxeA,...,Dh8R00m1wcCsUcm-Sflmlw,9vICKr25A3YG88Zr1wxDiQ,mGVVbBW0e6NVyNxHRstz_w,aM-FPdJicti_TzymjW3Usg,EwUahd4SYKdPH1p1Cpe5fg,2faLQhuvFdk-fHAg9lWXeA,C2Sq5A2g7a56wwiqrQ0ucA,mOnesB4IF9j6-ZmHoOHOig,TP0_HnsbEzhz6gMj9iyTFA,1diwr6ycCvtBXR5vmnmNGg
82,v1UzkU8lEWdjxq8byWFOKg,5_RnnLBKO-5vpb4iiFAvBQ,-L69Ix0-xX4BlHA61fGvrQ,2faLQhuvFdk-fHAg9lWXeA,P2bibaW6DSwnCpjVXbsZQQ,mOnesB4IF9j6-ZmHoOHOig,EwUahd4SYKdPH1p1Cpe5fg,dQ80ktWSRj5UIk96bRUmWw,KK-6rXbIo9B4-b_P5WO5Qg,Dh8R00m1wcCsUcm-Sflmlw,...,ezX80RabJ62B2lbKXCtg3g,-_GnwXmzC3DXsHR9nyaC2g,oug5bLTWP_YTtj1C3_X6Xw,eOnMGfoj3YOxGPDjrDE-sg,bZiIIUcpgxh8mpKMDhdqbA,9vICKr25A3YG88Zr1wxDiQ,KEfjH-T30yTvXssJLjcrlA,-oCYfB-JL3Yc_RECg4B1Qw,E9ysHn296GgE9KJg88t83w,Z-URyf2TJT1Ay-Sp1cVGOw
83,vJUTrk0-nm7JbNzPVcPo4A,vzSXdDYtnm_F_x1hY2UVVw,bZiIIUcpgxh8mpKMDhdqbA,4cgE4DUDSH3CjqsRwX_LOA,1diwr6ycCvtBXR5vmnmNGg,uv1Fl6F7I9KRONRCg2pKpQ,Z4CPbD4a-cR4JR8CH_c3PQ,g-6qv5f2Y4mVrsDD3JRkuQ,mOnesB4IF9j6-ZmHoOHOig,KEh3efWSOIzd8BvuW-X4-g,...,WtDOs3a6k_oPJmwiDh4qBQ,La_xmePMtWwjxKUqbGvqGQ,jREzLrIEkc4jQKLfYMJ0gg,oug5bLTWP_YTtj1C3_X6Xw,OlB0841vj4V7Wje5tTtLWA,Dh8R00m1wcCsUcm-Sflmlw,bWfbGdWSfRFLswxnoMt5oA,KEfjH-T30yTvXssJLjcrlA,ezX80RabJ62B2lbKXCtg3g,dQ80ktWSRj5UIk96bRUmWw
84,vzSXdDYtnm_F_x1hY2UVVw,vJUTrk0-nm7JbN

In [130]:
result_df[result_df.business_id == 'Caroline']['Top_1'].value_counts().idxmax()

"Phoebe's Diner"

In [135]:
k = grouped_rest[['business_id','stars','name']].merge(result_df)

In [136]:
grouped_rest[['business_id','categories','review_count','address']].merge(k).drop_duplicates()

,business_id,categories,review_count,address,stars,name,Top_0,Top_1,Top_2,Top_3,...,Top_10,Top_11,Top_12,Top_13,Top_14,Top_15,Top_16,Top_17,Top_18,Top_19
0,-1ShItlulHnBsoOQWnblzw,"Food Trucks, Mexican, Food, Restaurants",105,7309 McNeil Dr,2.0,Super Burrito at the Water Tank,NY_oCeK6mqVz-oJfbD0c4A,qDyZQ6mT1JajQ7_Yj7PeLw,4cgE4DUDSH3CjqsRwX_LOA,mOnesB4IF9j6-ZmHoOHOig,...,KEfjH-T30yTvXssJLjcrlA,mGVVbBW0e6NVyNxHRstz_w,jREzLrIEkc4jQKLfYMJ0gg,hkqFm02Y2ax0Pd1BdEyl1A,C0d5kzUx6C19mLcxQyhxCA,KEh3efWSOIzd8BvuW-X4-g,-oCYfB-JL3Yc_RECg4B1Qw,vJUTrk0-nm7JbNzPVcPo4A,_rt-Z934kfFzgG19nTrIcQ,E9ysHn296GgE9KJg88t83w
1,-IIvmjoEKa9Trhf4OxzJeA,"Bakeries, Food, Breakfast & Brunch, American (...",770,621 Congress Ave,5.0,Caroline,iCpyXPsePXB_UqT4_hVqBg,KEh3efWSOIzd8BvuW-X4-g,_HjtaT4TMHwOoGG_pZmk1A,JCgLWYjoPJlznzFEtnzqKg,...,Mvbfzj05yR5dH397_7XPQw,7CXy5h3u_aWsK9qov6B4rg,LPYACwYAeHJdQIpxEtJYvw,dQ80ktWSRj5UIk96bRUmWw,LUCmOKTK5Uh7eU84yJq3mQ,g-6qv5f2Y4mVrsDD3JRkuQ,GCgnLwwI4b9cOee-vtK1aQ,F6b_O4-wtd3956YOvAoerg,OlB0841vj4V7Wje5tTtLWA,aM-FPdJicti_TzymjW3Usg
2,-L69Ix0-xX4BlHA61fGvrQ,"Fast Food, Restaurants, Food, Juice Bars & Smo...",73,"8300 N Fm 620, Bldg H",5.0,Chicken Express,KEfjH-T30yTvXssJLjcrlA,KK-6rXbIo9B4-b_P5WO5Qg,ezX80RabJ62B2lbKXCtg3g,v1UzkU8lEWdjxq8byWFOKg,...,Z-URyf2TJT1Ay-Sp1cVGOw,IbuSlumTCNSD-ohMVHfWUA,9vICKr25A3YG88Zr1wxDiQ,4THc7ojs6ppNHKhLl74www,Dh8R00m1wcCsUcm-Sflmlw,Pkp824mx2LL93TqKOdfMyQ,i1GWpxJ1Phm2awhVaTkouA,Id7w_CCXqh7x1lC3U1-S5w,OlB0841vj4V7Wje5tTtLWA,bZiIIUcpgxh8mpKMDhdqbA
3,-_GnwXmzC3DXsHR9nyaC2g,"Restaurants, Mexican",1148,614 E Oltorf St,4.0,Curra's Grill,E9ysHn296GgE9KJg88t83w,oug5bLTWP_YTtj1C3_X6Xw,_Rrpjd7Z7isi8QFAv3kQ2g,4cgE4DUDSH3CjqsRwX_LOA,...,2faLQhuvFdk-fHAg9lWXeA,OlB0841vj4V7Wje5tTtLWA,g-6qv5f2Y4mVrsDD3JRkuQ,9vICKr25A3YG88Zr1wxDiQ,tmvswVDr4n56EjWq_RptBA,NY_oCeK6mqVz-oJfbD0c4A,-oCYfB-JL3Yc_RECg4B1Qw,pc1PkDBggUSNU73jONdqHw,KEfjH-T30yTvXssJLjcrlA,WtDOs3a6k_oPJmwiDh4qBQ
4,-oCYfB-JL3Yc_RECg4B1Qw,"Wedding Chapels, Donuts, Food, Bakeries, Event...",1809,212 E 6th St,3.0,Voodoo Doughnut - 6th Street,_Rrpjd7Z7isi8QFAv3kQ2g,o8DMdE-VAiUydUBJ2E6MZA,dQ80ktWSRj5UIk96bRUmWw,Dh8R00m1wcCsUcm-Sflmlw,...,mOnesB4IF9j6-ZmHoOHOig,g-6qv5f2Y4mVrsDD3JRkuQ,khghDJpPqW5hG4IXr8sF4A,gcAXWYT_yoJCE5cAAeaHxQ,9vICKr25A3YG88Zr1wxDiQ,-_GnwXmzC3DXsHR9nyaC2g,bZiIIUcpgxh8mpKMDhdqbA,4cgE4DUDSH3CjqsRwX_LOA,_jYt69Zx1SUo_V9z0m7Ljg,oug5bLTWP_YTtj1C3_X6Xw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,uv1Fl6F7I9KRONRCg2pKpQ,"Restaurants, Mexican, Hawaiian, Seafood, Food,...",114,1722 South Congress,3.0,Wahoo's Fish Tacos,bZiIIUcpgxh8mpKMDhdqbA,LPYACwYAeHJdQIpxEtJYvw,pc1PkDBggUSNU73jONdqHw,tmvswVDr4n56EjWq_RptBA,...,Dh8R00m1wcCsUcm-Sflmlw,9vICKr25A3YG88Zr1wxDiQ,mGVVbBW0e6NVyNxHRstz_w,aM-FPdJicti_TzymjW3Usg,EwUahd4SYKdPH1p1Cpe5fg,2faLQhuvFdk-fHAg9lWXeA,C2Sq5A2g7a56wwiqrQ0ucA,mOnesB4IF9j6-ZmHoOHOig,TP0_HnsbEzhz6gMj9iyTFA,1diwr6ycCvtBXR5vmnmNGg
82,v1UzkU8lEWdjxq8byWFOKg,"Soul Food, Southern, Comfort Food, Chicken Sho...",2490,117 San Jacinto Blvd,5.0,Gus's World Famous Fried Chicken,5_RnnLBKO-5vpb4iiFAvBQ,-L69Ix0-xX4BlHA61fGvrQ,2faLQhuvFdk-fHAg9lWXeA,P2bibaW6DSwnCpjVXbsZQQ,...,ezX80RabJ62B2lbKXCtg3g,-_GnwXmzC3DXsHR9nyaC2g,oug5bLTWP_YTtj1C3_X6Xw,eOnMGfoj3YOxGPDjrDE-sg,bZiIIUcpgxh8mpKMDhdqbA,9vICKr25A3YG88Zr1wxDiQ,KEfjH-T30yTvXssJLjcrlA,-oCYfB-JL3Yc_RECg4B1Qw,E9ysHn296GgE9KJg88t83w,Z-URyf2TJT1Ay-Sp1cVGOw
83,vJUTrk0-nm7JbNzPVcPo4A,"Japanese, Sushi Bars, Restaurants",250,2900 W Anderson Ln,4.0,Zen Japanese Food Fast,vzSXdDYtnm_F_x1hY2UVVw,bZiIIUcpgxh8mpKMDhdqbA,4cgE4DUDSH3CjqsRwX_LOA,1diwr6ycCvtBXR5vmnmNGg,...,WtDOs3a6k_oPJmwiDh4qBQ,La_xmePMtWwjxKUqbGvqGQ,jREzLrIEkc4jQKLfYMJ0gg,oug5bLTWP_YTtj1C3_X6Xw,OlB0841vj4V7Wje5tTtLWA,Dh8R00m1wcCsUcm-Sflmlw,bWfbGdWSfRFLswxnoMt5oA,KEfjH-T30yTvXssJLjcrlA,ezX80RabJ62B2lbKXCtg3g,dQ80ktWSRj5UIk96bRUmWw
84,vzSXdDYtnm_F_x1hY2UVVw,"Noodles, Restaurants, Chinese",471,8650 Spicewood Springs Rd,4.0,Chen's Noodle House,vJUTrk0-nm7JbNzPVcPo4A,La_xmePMtWwjxKUqbGvqGQ,_jYt69Zx1SUo_V9z0m7Ljg,-oCYfB-JL3Yc_RECg4B